In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
#Importing Dataset
df=pd.read_excel("jamadoba.xlsx")

In [3]:
#Preprocessing
df=df.rename(columns={"Resistance in Murgue":"Resistance","Quantity in Cu.m/min":"Quantity","Pressure Drop in mmwg":"Pressure"})
df['Resistance']=df['Resistance'].replace('*********',np.NaN)
df['Resistance']=df['Resistance'].replace(np.NaN,(df['Pressure']/np.square(df['Quantity']/60))*1000)
df['Q'] = np.square(df['Quantity']/60)
#df['Quantity']=round(df['Quantity'],-2)
df.head(10)

,Branch,J1,J2,Resistance,Quantity,Pressure,Q
0,1,1,6,1.000,1468.000,0.599,598.617778
1,2,142,332,1.000,150.000,0.006,6.250000
2,3,140,333,1.000,50.000,0.001,0.694444
3,4,502,1,10.000,1361.000,5.145,514.533611
4,5,6,1,1.565,8081.673,28.393,18142.621800
5,6,1,2,0.203,4464.288,1.124,5536.074263
6,7,2,3,0.457,4220.808,2.264,4948.672270
7,8,3,140,0.319,4220.808,1.579,4948.672270
8,9,140,141,0.784,4170.808,3.788,4832.122048
9,10,141,142,0.616,4170.808,2.977,4832.122048


In [4]:
x = df.iloc[:,[3,4]].values
y = df.iloc[:,[5]].values

In [5]:
from sklearn.model_selection import KFold,cross_val_score

In [9]:
kf = KFold(n_splits=6,shuffle=False)

In [10]:
for train_index, test_index in kf.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252
 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
 271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
 289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
 343 344 345 346 347 348 349 350 351 352 353

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

regressorRes = RandomForestRegressor(n_estimators = 500, random_state = 0)

for test,train in kf.split(x):
    regressorRes.fit(x[train],y[train].ravel())
    y_pred=regressorRes.predict(x[test])
    p=regressorRes.predict(x[train])
    rmse_test = sqrt(mean_squared_error(y[test], y_pred))
    print("Rmse test=",rmse_test)
    rmse_train = sqrt(mean_squared_error(y[train], p))
    print("Rmse train=",rmse_train)




Rmse test= 5.5609659977671475
Rmse train= 3.443447293009778
Rmse test= 5.475805637905797
Rmse train= 3.3702319602462576
Rmse test= 5.3529846848165645
Rmse train= 3.304677131651087
Rmse test= 5.684457568208492
Rmse train= 3.5741990798960277
Rmse test= 5.724213892639449
Rmse train= 3.2979383360662604
Rmse test= 4.689894418940549
Rmse train= 4.163346212477353


In [23]:
y_pred_check = regressorRes.predict(np.array([(900,477)],ndmin=2))
y_pred_check

array([62.287516])